# Pre-training GPT2, Step1 制备文本数据集

本次实验的代码参考了书籍《从零构建大模型》

English version: "Build a Large Language Model From Scratch" 

作者：[美] 塞巴斯蒂安·拉施卡 [Sebastian Raschka]

译者：覃立波 冯骁骋 刘乾

出版社：人民邮电出版社有限公司 

出版时间：2025-04-01 

ISBN：9787115666000

书籍代码库：

https://github.com/rasbt/LLMs-from-scratch/tree/main




## (1) 下载开源的文本数据
我们的目标是将这篇包含20479个字符的短篇小说分割为独立的单词和特殊字符，以便在后续章节中将它们转换为嵌入向量，进而用于大语言模型训练。注意　构建大语言模型需要处理数百万篇文章和成千上万本图书，通常多达数千兆字节(GB)数据。不过，出于教学目的，使用小规模的文本样本（如一本书）就足以说明文本处理步骤的核心思想，并且可以在合理时间内在消费级硬件上完成运行。

In [ ]:

#下载简化数据集的文本（一本开源的英文短篇小说 verdict），设置下载地址，查看本地路径，打印输出小说对应的前100个字符。
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

#print current working directory
import os
print('vertex is downloaded at', os.getcwd())

# 通过Python读取短篇小说TheVerdict作为文本样本,
#打印(print)命令首先输出该文件的字符总数，然后展示文件的前100个字符作为内容示例：
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
print("Total number of character:", len(raw_text))
print(raw_text[:100])

/Users/yizhou/teaching/人工智能导论/code
Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g


## (2) 选择并测试分词器
接下来，我们将把这些词元从Python字符串转换为整数表示，以生成词元token ID。这一过程是将词元ID转换为嵌入向量前的必经步骤。我们将使用现有的Python开源库tiktoken，它基于Rust的源代码非常高效地实现了BPE算法。与其他Python库类似，可以通过Python的pip安装器从终端安装tiktoken库, 如:

pip install tiktoken

In [5]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version("tiktoken"))
tokenizer = tiktoken.get_encoding("gpt2")
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of some unknown Place."
)

#tittoken通过encode函数实现对文本的分词和词元id转换
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

#反过来，通过decode函数实现从数字id到文本词元的复原
strings = tokenizer.decode(integers)
print(strings)

tiktoken version: 0.11.0
[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 6439, 8474, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof some unknown Place.


## (3)制备训练所需的训练集和验证集。
训练和验证数据包括 输入的文本‘序列’ x 和对应的下一个文本‘单词’ target。
训练数据的总数大约是验证数据的4倍。

输入的文本序列和对应的目标文本，利用滑窗的形式从小说的文本中采取。

下图采用的是步长为1的滑窗（输入x和输出y序列仅有一个单词不同）


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch02_compressed/13.webp?123" width="500px">


In [2]:
#定义一个数据集，用input_ids和target_ids分别存储输入和目标文本序列的词元id
#用max_length和stride来控制输入和目标文本序列的长度，以及文本跳跃的步长
#用allowed_special={"<|endoftext|>"}来处理特殊字符
#用torch.tensor()将词元id转换为张量
#用__len__()和__getitem__()来实现数据集的索引和获取
#用DataLoader来实现数据集的批量加载

from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [3]:
#定义一个函数，用create_dataloader_v1来创建数据加载器
#其中输入参数txt是文本，batch_size是批量大小，max_length是输入和目标文本序列的长度，
# stride是文本跳跃的步长，shuffle是是否打乱数据，drop_last是是否丢弃最后一个不完整的批次，
# num_workers是数据加载的线程数
#用tokenizer来分词
#用GPTDatasetV1来创建数据集
#用DataLoader来创建数据加载器
#用return dataloader来返回数据加载器

def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [10]:
#测试数据集访问。
import torch
import tiktoken
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter  = iter(dataloader) # 创建数据迭代器
first_input_batch, first_target_batch = next(data_iter) # 用next函数来顺序访问每一个batch
print('first_input_batch:', first_input_batch)
print('first_target_batch:', first_target_batch)


#打印第一个batch的输入和目标文本序列
print(tokenizer.decode(first_input_batch[0].numpy()))
print(tokenizer.decode(first_target_batch[0].numpy()))

first_input_batch: tensor([[  40,  367, 2885, 1464]])
first_target_batch: tensor([[ 367, 2885, 1464, 1807]])
I HAD always
 HAD always thought


## (4)实验任务一， 通过调整哪个参数可以使得两个输入文本的序列之间不存在重合的文字？

In [ ]:
#请在此格写出代码，并观察相邻两个输入文本序列的token ids和对应的文本。